In [181]:
import pandas as pd
import json
import csv
import numpy as np
import psycopg2
import ast
from sqlalchemy import create_engine, text

In [182]:
pd.set_option('display.max_columns', None)
property_df = pd.read_csv('detailed_compounds_data.csv')
property_df.head()

,propertyID,propertyName,propertyTypeID,propertyTypeName,compoundID,areaID,develperID,finishing,min_unit_area,max_unit_area,min_price,max_price,max_installment_years,min_installments,min_down_payment,number_of_bathrooms,number_of_bedrooms,min_ready_by,sponsored,new_property,company,resale,financing,type,has_offers,offer_title,limited_time_offer,in_quick_search,recommended,manual_ranking,completeness_score,ranking_type,recommended_financing,property_ranking,compound_ranking,tags
0,35502,Millennial L Park middle for sale in Mountain ...,21,Apartment,195,2,6,not_finished,135.0,135.0,7584822.0,7584822.0,9.0,63206,758482,3.0,3.0,2027-01-01,0,False,NaN,False,False,NaN,False,"5% Down Payment , instalments up to 9 years",False,2,NaN,NaN,1,fast_moving,14842.0,1.0,9.0,[]
1,35495,Millennial A Park middle for sale in Mountain ...,21,Apartment,195,2,6,not_finished,135.0,135.0,8015714.0,8015714.0,9.0,66797,801571,3.0,3.0,2027-01-01,0,False,NaN,False,False,NaN,False,"5% Down Payment , instalments up to 9 years",False,2,NaN,NaN,1,fast_moving,14842.0,4.0,9.0,[]
2,35497,Millennial L Park corner for sale in Mountain ...,21,Apartment,195,2,6,not_finished,145.0,145.0,8219763.0,8219763.0,9.0,68497,821976,3.0,3.0,2027-01-01,0,False,NaN,False,False,NaN,False,"5% Down Payment , instalments up to 9 years",False,2,NaN,NaN,1,fast_moving,14842.0,2.0,9.0,[]
3,35490,Millennial A Park corner for sale in Mountain ...,21,Apartment,195,2,6,not_finished,130.0,130.0,8505952.0,8505952.0,9.0,70882,850595,3.0,3.0,2027-01-01,0,False,NaN,False,False,NaN,False,"5% Down Payment , instalments up to 9 years",False,2,NaN,NaN,1,fast_moving,14842.0,7.0,9.0,[]
4,35487,IVilla Park middle for sale in Mountain View I...,3,Duplex,195,2,6,not_finished,270.0,270.0,13776405.0,13776405.0,9.0,114803,1377640,3.0,3.0,2027-01-01,0,False,NaN,False,False,NaN,False,"5% Down Payment , instalments up to 9 years",False,2,NaN,NaN,1,fast_moving,14842.0,3.0,9.0,[]


In [183]:
property_table = property_df[['propertyID','propertyName', 'propertyTypeID',
                              'compoundID', 'areaID', 'develperID', 'finishing',
                              'max_unit_area', 'min_price', 'max_price', 'max_installment_years',
                              'min_installments', 'min_down_payment', 'number_of_bathrooms',
                                'number_of_bedrooms', 'min_ready_by', 'sponsored',
                                'new_property', 'resale', 'financing',
                                 'has_offers', 'offer_title', 'limited_time_offer',
                                'in_quick_search', 'manual_ranking', 'completeness_score',
                                'ranking_type', 'recommended_financing', 'property_ranking',
                                'compound_ranking', 'tags']]

In [184]:
# connnect to db 
# Replace these with your database details
dbname = "nawy"
user = "nady"
password = "admin"
host = "172.20.0.3"
port = "5432"


# Establish a connection to the database
try:
    connection = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )

except psycopg2.Error as e:
    print("Error connecting to the database:", e)
    exit()


# Create SQLalchemy engine
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")

In [185]:
column_mapping = {
    'compoundID': 'compound_id',
    'develperID':'developer_id',
    'max_unit_area':'unit_area',
    'tags':'tag'}
property_table.rename(columns=column_mapping, inplace=True)

/tmp/ipykernel_6001/4030140588.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_table.rename(columns=column_mapping, inplace=True)


In [186]:
#check df has the same db tables column name
columns = sorted(property_table.columns) 
# Query to retrieve column names
with engine.connect() as connection:
    query = text("SELECT column_name FROM information_schema.columns WHERE table_name = 'property' ORDER BY column_name ASC;")
    results = connection.execute(query)

    # Fetch the results
    column_names = results.fetchall()

# Display the column names
for row, column_name in enumerate(column_names):
    results = column_name[0] == columns[row]
    if results == True:
        pass
    else:
        print(f"{column_name[0]} vs {columns[row]}")

In [187]:
# casting diff data types
#property_table['financing'] = property_table['financing'].astype(bool)

# Convert 'min_ready_by' column to datetime
property_table['min_ready_by'] = pd.to_datetime(property_table['min_ready_by'], errors='coerce')

# Convert 'number_of_bathrooms' and 'number_of_bedrooms' columns to integers, handling errors
property_table['number_of_bathrooms'] = pd.to_numeric(property_table['number_of_bathrooms'], errors='coerce').astype('Int64')
property_table['number_of_bedrooms'] = pd.to_numeric(property_table['number_of_bedrooms'], errors='coerce').astype('Int64')


/tmp/ipykernel_6001/2044599106.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_table['min_ready_by'] = pd.to_datetime(property_table['min_ready_by'], errors='coerce')
/tmp/ipykernel_6001/2044599106.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_table['number_of_bathrooms'] = pd.to_numeric(property_table['number_of_bathrooms'], errors='coerce').astype('Int64')
/tmp/ipykernel_6001/2044599106.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [188]:
## handling tag column
#property_table.ilco[0,'tag']
property_table['tag'] = property_table['tag'].apply(lambda x: ast.literal_eval(x)[0] if ast.literal_eval(x) else '')

/tmp/ipykernel_6001/265542195.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_table['tag'] = property_table['tag'].apply(lambda x: ast.literal_eval(x)[0] if ast.literal_eval(x) else '')


In [189]:
## match column datatypes in db table with column datatypes in df table
column_types = property_table.dtypes.sort_index()
column_types
with engine.connect() as connection:
    query = text(
        f"select column_name, data_type from information_schema.columns where table_name='property' ORDER BY column_name"
    )
    # Excute the query
    results = connection.execute(query)

    #fetch the result
    column_info = results.fetchall()
for row, column_ in enumerate(column_info):
    results = column_[0] == columns[row]
    if results == True:
        print(f"{column_[0]:<30} --> {column_[1]:<20}, :: {column_types[row]}")
    else:
        print(f"{column_name[0]} vs {columns[row]}")

areaID                         --> integer             , :: int64
completeness_score             --> integer             , :: int64
compound_id                    --> integer             , :: int64
compound_ranking               --> double precision    , :: float64
developer_id                   --> integer             , :: int64
financing                      --> boolean             , :: bool
finishing                      --> character varying   , :: object
has_offers                     --> boolean             , :: bool
in_quick_search                --> integer             , :: int64
limited_time_offer             --> boolean             , :: bool
manual_ranking                 --> double precision    , :: float64
max_installment_years          --> double precision    , :: float64
max_price                      --> numeric             , :: float64
min_down_payment               --> numeric             , :: int64
min_installments               --> integer             , :: int64
min_

/tmp/ipykernel_6001/2682882931.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"{column_[0]:<30} --> {column_[1]:<20}, :: {column_types[row]}")


In [190]:
### drop duplicate properties
property_table = property_table.drop_duplicates(subset='propertyID', keep='first')

# I have drop all proberty with compound which not refrence any compound in compound table
# I have to left megre the two tbale here before inserting to find missed compounIds
property_table = property_table[~property_table['compound_id'].isin([337, 822, 663, 717, 199, 654, 260])]

In [191]:
# insert data to database
property_table.to_sql('property', engine, if_exists='append', index=False)

770

## load data to property_type table

In [192]:
pd.set_option('display.max_columns', None)
property_df = pd.read_csv('detailed_compounds_data.csv')
property_df.head()

,propertyID,propertyName,propertyTypeID,propertyTypeName,compoundID,areaID,develperID,finishing,min_unit_area,max_unit_area,min_price,max_price,max_installment_years,min_installments,min_down_payment,number_of_bathrooms,number_of_bedrooms,min_ready_by,sponsored,new_property,company,resale,financing,type,has_offers,offer_title,limited_time_offer,in_quick_search,recommended,manual_ranking,completeness_score,ranking_type,recommended_financing,property_ranking,compound_ranking,tags
0,35502,Millennial L Park middle for sale in Mountain ...,21,Apartment,195,2,6,not_finished,135.0,135.0,7584822.0,7584822.0,9.0,63206,758482,3.0,3.0,2027-01-01,0,False,NaN,False,False,NaN,False,"5% Down Payment , instalments up to 9 years",False,2,NaN,NaN,1,fast_moving,14842.0,1.0,9.0,[]
1,35495,Millennial A Park middle for sale in Mountain ...,21,Apartment,195,2,6,not_finished,135.0,135.0,8015714.0,8015714.0,9.0,66797,801571,3.0,3.0,2027-01-01,0,False,NaN,False,False,NaN,False,"5% Down Payment , instalments up to 9 years",False,2,NaN,NaN,1,fast_moving,14842.0,4.0,9.0,[]
2,35497,Millennial L Park corner for sale in Mountain ...,21,Apartment,195,2,6,not_finished,145.0,145.0,8219763.0,8219763.0,9.0,68497,821976,3.0,3.0,2027-01-01,0,False,NaN,False,False,NaN,False,"5% Down Payment , instalments up to 9 years",False,2,NaN,NaN,1,fast_moving,14842.0,2.0,9.0,[]
3,35490,Millennial A Park corner for sale in Mountain ...,21,Apartment,195,2,6,not_finished,130.0,130.0,8505952.0,8505952.0,9.0,70882,850595,3.0,3.0,2027-01-01,0,False,NaN,False,False,NaN,False,"5% Down Payment , instalments up to 9 years",False,2,NaN,NaN,1,fast_moving,14842.0,7.0,9.0,[]
4,35487,IVilla Park middle for sale in Mountain View I...,3,Duplex,195,2,6,not_finished,270.0,270.0,13776405.0,13776405.0,9.0,114803,1377640,3.0,3.0,2027-01-01,0,False,NaN,False,False,NaN,False,"5% Down Payment , instalments up to 9 years",False,2,NaN,NaN,1,fast_moving,14842.0,3.0,9.0,[]


In [195]:
property_type = property_df[['propertyTypeID', 'propertyTypeName', 'propertyID']]
property_type.head()

,propertyTypeID,propertyTypeName,propertyID
0,21,Apartment,35502
1,21,Apartment,35495
2,21,Apartment,35497
3,21,Apartment,35490
4,3,Duplex,35487


In [204]:
## retrieve all propertyIDs in property_atable
# Query to retrieve column names
with engine.connect() as connection:
    query = text("""SELECT "propertyID" from property;""")
    results = connection.execute(query)
    # Fetch and print the propertyID values
    property_ids = [row[0] for row in results.fetchall()]
print(property_ids)

[380, 382, 397, 398, 399, 403, 547, 548, 549, 551, 958, 1121, 1246, 1350, 1351, 1352, 1619, 1620, 1621, 1625, 1626, 1627, 1628, 1942, 1955, 1963, 2075, 2076, 2112, 2113, 2115, 2118, 2119, 2120, 2131, 2132, 2191, 2192, 2193, 2194, 2195, 2198, 2250, 2251, 2385, 2386, 2387, 2478, 2480, 2721, 2774, 2840, 2842, 2916, 3121, 3245, 3324, 3327, 3333, 3334, 3338, 3462, 3517, 3643, 3671, 3721, 3722, 3724, 3725, 3813, 3858, 3867, 3870, 4026, 4031, 4062, 4066, 4095, 4096, 4097, 4098, 4099, 4102, 4103, 4104, 4105, 4106, 4107, 4109, 4127, 4128, 4131, 4132, 4140, 4141, 4142, 4144, 4146, 4147, 4148, 4151, 4153, 4156, 4158, 4159, 4172, 4299, 4300, 4307, 4308, 4309, 4312, 4320, 4322, 4324, 4325, 4326, 4327, 4357, 4372, 4375, 4385, 4386, 4387, 4388, 4389, 4390, 4391, 4461, 4463, 4467, 4545, 4590, 4663, 4664, 4668, 4686, 4687, 4700, 4753, 4763, 4765, 5543, 5544, 5564, 5565, 5683, 5710, 5810, 5880, 5907, 5958, 5959, 5962, 5964, 5965, 5966, 5969, 5971, 5972, 5974, 5997, 6232, 6307, 6695, 6730, 6758, 7037, 70

In [208]:
property_type = property_type[property_type['propertyID'].isin(property_ids)]
property_type.columns

Index(['propertyTypeID', 'propertyTypeName', 'propertyID'], dtype='object')

In [209]:
# adjust column naming
column_mapping = {
    'propertyTypeID': 'id',
    'propertyTypeName':'name',
    'propertyID':'propertyID'}
property_type.rename(columns=column_mapping, inplace=True)

In [211]:
# check column datatypes
column_types = property_type.dtypes.sort_index()
column_types

id             int64
name          object
propertyID     int64
dtype: object

In [215]:
# insert data to database
property_type.drop_duplicates(subset=['id'], keep='first', inplace=True)
property_type.to_sql('property_type', engine, if_exists='append', index=False)

14